In [1]:
import os
import sys
import importlib.util

sys.path.append('./src/')

from helpers.snakemake import nested_dict_update, get_git_revision_hash
from default_net_params import params as net_params
from data_preprocessing.cytoarchitecture import NeuronNumbers
from data_preprocessing.connectivity import SynapseNumbers
from network import Network

In [2]:
# Set base path
base_path = os.getcwd()
experiments_path = os.path.join(base_path, 'experiments')

In [3]:
# Load script from specified path sys.argv[1] (snakemake {input})
# Does nothing else than `import path/to/script as exp` would do
experiment_file = './experiments/experiment_stabilized_groundstate.py'
# experiment_file = './experiments/experiment_stabilized_bestfit.py'
conf_name, _ = os.path.splitext(os.path.basename(experiment_file))
conf_path = os.path.join(os.getcwd(), experiment_file)
spec = importlib.util.spec_from_file_location(conf_name, conf_path)
exp = importlib.util.module_from_spec(spec)
spec.loader.exec_module(exp)

In [4]:
# Update default parameters with parameters specified in exp
nested_dict_update(net_params, exp.net_params)
outpath = net_params['outpath']

# Set simulation parameters
exp.sim_params['t_sim'] = 2000.0

# Set plotting parameters
exp.ana_params['plotRasterArea']['low'] = 1500
exp.ana_params['plotRasterArea']['high'] = 2000
exp.ana_params['plotRasterArea']['fraction'] = 0.5

# Set scaling parameters
scaling_factor = 0.005
net_params['N_scaling'] = scaling_factor
net_params['K_scaling'] = scaling_factor

# Mean fullscale firing rate
net_params['fullscale_rates'] = './simulated_data/base_theory_rates.pkl'

In [5]:
import pandas as pd
pd.read_pickle(net_params['fullscale_rates'])

area                layer   population
bankssts            II/III  E             0.059896
                            I             0.357734
                    IV      E             1.364229
                            I             2.078277
                    V       E             0.240457
                                            ...   
transversetemporal  IV      I             1.322175
                    V       E             0.662288
                            I             0.999459
                    VI      E             1.332735
                            I             4.964037
Length: 254, dtype: float64

In [6]:
# Update default parameters with parameters specified in exp
nested_dict_update(net_params, exp.net_params)

### Creating the network

Here the network is created using the `NeuronNumbers` and `SynapseNumbers` classes. The `NeuronNumbers` class calculates the number of neurons in the network. The `SynapseNumbers` class calculates the number of synapses in the network and the synaptic weights.

In [7]:
# Create Network class
NN = NeuronNumbers(
    surface_area=net_params['surface_area'],
    **net_params['cytoarchitecture_params']
)

Dropping layer IV in caudalmiddlefrontal because it contains 3375 < 5000 neurons.
Dropping layer IV in lateraloccipital because it contains 3000 < 5000 neurons.
Dropping layer VI in parahippocampal because it contains 3200 < 5000 neurons.
Dropping layer IV in parsorbitalis because it contains 625 < 5000 neurons.


In [8]:
# Create Synapse class
SN = SynapseNumbers(
    NN=NN,
    **net_params['predictive_connectomic_params']
)

In [9]:
# Create Network
net = Network(NN, SN, net_params)

# Export the network
net.dump(outpath)

# Get the network hash
net_hash = net.getHash()

# To access net object key, use net.net.keys()
# Ex.: to access neuron numbers, use net.net['neuron_numbers']

### Running the simulation

In [10]:
from default_sim_params import params as sim_params
from network import networkDictFromDump
from simulation import Simulation

[INFO] [2024.10.9 18:30:46 /home/conda/feedstock_root/build_artifacts/nest-simulator_1636957195130/work/nestkernel/rng_manager.cpp:217 @ Network::create_rngs_] : Creating default RNGs
[INFO] [2024.10.9 18:30:46 /home/conda/feedstock_root/build_artifacts/nest-simulator_1636957195130/work/nestkernel/rng_manager.cpp:260 @ Network::create_grng_] : Creating new default global RNG

              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: nest-2.20.2
 Built: Nov 15 2021 06:23:02

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [11]:
# Update default simulation parameters with parameters specified in exp
nested_dict_update(sim_params, exp.sim_params)

In [12]:
# Read network dict
net_folder = os.path.join(outpath, net_hash)
net_dict = networkDictFromDump(net_folder)

/home/shimoura/Documents/GitHub/humam-add-downscaling-factor/./src/network.py:874: YAMLLoadWarning:calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
/home/shimoura/Documents/GitHub/humam-add-downscaling-factor/./src/network.py:879: YAMLLoadWarning:calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
/home/shimoura/Documents/GitHub/humam-add-downscaling-factor/./src/network.py:884: YAMLLoadWarning:calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
/home/shimoura/Documents/GitHub/humam-add-downscaling-factor/./src/network.py:889: YAMLLoadWarning:calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


In [13]:
# Create Simulation class, export it and calculate the hash
sim = Simulation(sim_params, net_dict)
sim.dump(net_folder)
sim_hash = sim.getHash()

In [15]:
# Set output directory according to hashes, instantiate the Network
# and run the simulation
data_path = os.path.join(outpath, net_hash, sim_hash)
num_threads = int(4)
sim.setup(data_path, num_threads)
sim.simulate()

data directory already exists
Data will be written to /home/shimoura/Documents/GitHub/humam-add-downscaling-factor/out/5483b41ed6786feba1c19e51c3c6de42/57337a790beee98a454ec94c51878951
Master seed: 2106 
Number of total processes: 4
Seeds for random number generators of virtual processes: [2111, 2112, 2113, 2114]
Global random number generator seed: 2110

Oct 09 18:30:47 ModelManager::clear_models_ [Info]: 
    Models will be cleared and parameters reset.

Oct 09 18:30:47 Network::create_rngs_ [Info]: 
    Deleting existing random number generators

Oct 09 18:30:47 Network::create_rngs_ [Info]: 
    Creating default RNGs

Oct 09 18:30:47 Network::create_grng_ [Info]: 
    Creating new default global RNG

Oct 09 18:30:47 ModelManager::clear_models_ [Info]: 
    Models will be cleared and parameters reset.

Oct 09 18:30:47 Network::create_rngs_ [Info]: 
    Deleting existing random number generators

Oct 09 18:30:47 Network::create_rngs_ [Info]: 
    Creating default RNGs

Oct 09 18:30:4

/home/shimoura/miniconda3/envs/humam/lib64/python3.9/site-packages/nest/lib/hl_api_helper.py:127: UserWarning:
GetNodes is deprecated and will be removed in NEST 3.0. Use             GIDCollection instead.


Memory on rank 0 after creating populations: 2191.79MB
Spike detectors created
Poisson background input created
Single spike input created
Connections are established
Connected all of area bankssts, layer II/III and population E on rank 0. Memory: 2191.98 MB.
Connected all of area bankssts, layer II/III and population I on rank 0. Memory: 2191.98 MB.
Connected all of area bankssts, layer IV and population E on rank 0. Memory: 2191.98 MB.
Connected all of area bankssts, layer IV and population I on rank 0. Memory: 2191.98 MB.
Connected all of area bankssts, layer V and population E on rank 0. Memory: 2191.98 MB.
Connected all of area bankssts, layer V and population I on rank 0. Memory: 2191.98 MB.
Connected all of area bankssts, layer VI and population E on rank 0. Memory: 2191.98 MB.
Connected all of area bankssts, layer VI and population I on rank 0. Memory: 2191.98 MB.
Connected all of area caudalanteriorcingulate, layer II/III and population E on rank 0. Memory: 2191.98 MB.
Connect

### Analysis

In [16]:
from simulation import simulationDictFromDump
from analysis import Analysis
from default_ana_params import params as ana_params

In [17]:
# Update default parameters with parameters specified in exp
nested_dict_update(net_params, exp.net_params)
nested_dict_update(ana_params, exp.ana_params)

# Read simulation dict
sim_folder = os.path.join(outpath, net_hash, sim_hash)
sim_dict = simulationDictFromDump(sim_folder)

# Create Analysis class and export it
ana = Analysis(ana_params, net_dict, sim_dict, sim_folder, base_path)
ana.dump(sim_folder)

# Do the analysis
# ana.fullAnalysis()

ana.popGids = ana._readPopGids()
ana.spikes = ana._readSpikes()
for area in net.net['area_list']:
    ana.plotRasterArea(area)

/home/shimoura/Documents/GitHub/humam-add-downscaling-factor/./src/simulation.py:553: YAMLLoadWarning:calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


Results will be written to /home/shimoura/Documents/GitHub/humam-add-downscaling-factor/out/5483b41ed6786feba1c19e51c3c6de42/57337a790beee98a454ec94c51878951/d0dead9a28c793771d74dff073828d66/plots
Plots will be written to /home/shimoura/Documents/GitHub/humam-add-downscaling-factor/out/5483b41ed6786feba1c19e51c3c6de42/57337a790beee98a454ec94c51878951/d0dead9a28c793771d74dff073828d66/plots
_readPopGids took 0.008 s
Loading SpikeTrains from gdf
_readSpikes took 15.885 s
